## Analiza danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne importy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Wczytanie danych

In [ ]:
def get_data():
    os.chdir('..')
    os.chdir('Zadanie1\\czesc4')
    read = []
    for filename in os.listdir():
        df = pd.read_csv(filename)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        read.append(df)
    os.chdir('..')
    os.chdir('..')
    os.chdir('Zadanie2')
    return read

data = get_data()
part6 = data[5]

## <center>Część 1</center>
### Wyznacz średnią kroczącą i odchylenie standardowe kroczące (np. 7-dniowe), aby zobaczyć, jak te zjawiska zmieniają się w czasie. Obliczenia wykonaj dla:


### 1.1. liczby nowych zachorowań,

### dla najbogatszego kraju z każdego kontynentu

In [ ]:
countries = part6[['country_name', 'gdp_usd', 'Continent']].sort_values(by='gdp_usd', ascending=False).drop_duplicates().groupby('Continent')
continents = part6['Continent'].unique()
choose_countries = np.array([countries.get_group(continent)['country_name'][:1].values for continent in continents])
choose_countries = choose_countries.reshape(-1)

### Średnia ruchoma i odchylenie standardowe

In [ ]:
def calculate_moving_avg_std(ylabel, column):
    fig, ax = plt.subplots(nrows=3, ncols=2, sharex=True, sharey=True, figsize=(20, 12))
    fig.suptitle('Średnia i ochylenie standardowe kroczące 7-dniowe', fontsize=20)
    for i, country_name in enumerate(choose_countries):
        x, y = divmod(i, 2)
        mean_data = part6.loc[part6['country_name'] == country_name, column].rolling(window=7).mean().bfill()
        std_data = part6.loc[part6['country_name'] == country_name, column].rolling(window=7).std().bfill()
        sns.lineplot(y=mean_data, x=part6['date'], ax=ax[x][y], color='red', label='średnia krocząca')
        sns.lineplot(y=std_data, x=part6['date'], ax=ax[x][y], color='green', label='odchylenie standardowe kroczące')
        ax[x][y].set_title(country_name)
        ax[x][y].set_xlabel('Date')
        ax[x][y].set_ylabel(ylabel)
        ax[x][y].legend(loc='best')


calculate_moving_avg_std('Liczba nowych zachorowań', 'daily_confirmed')

### 1.2. liczby nowych śmierci,

In [ ]:
calculate_moving_avg_std('Liczba nowych śmierci', 'daily_deceased')

### 1.3. liczby nowych szczepień,

In [ ]:
calculate_moving_avg_std('Liczba nowych szczepień', 'daily_persons_vaccinated')

### 1.4 liczba nowych zachorowań mężczyzn,

In [ ]:
calculate_moving_avg_std('Liczba nowych zachorowań mężczyzn', 'daily_confirmed_male')

### 1.4 liczba nowych śmierci mężczyzn

In [ ]:
calculate_moving_avg_std('Liczba nowych śmierci mężczyzn', 'daily_deceased_male')

## <center>Część 2</center>
### Wykorzystaj do analizy trendów analizę szeregów czasowych (metoda średniej ruchomej czy inne modele autoregresyjne), która umożliwi zbadanie 5 przypadków rozważanych w części 1 poziomu 3. Przeanalizuj otrzymane wyniki.

## <center>Cześć 3</center>
### Przygotuj dane treningowe w interesującym Cię okresie czasu (np. druga połowa 2020 roku i pierwsza połowa 2021 roku, tj. 52 tygodnie = X) oraz ewentualne dane testowe. Możesz przefiltrować dane także po innych kryteriach, jeżeli uznasz to za potrzebne. Zastosuj analizę regresji, aby przewidzieć wartości w kolejnych X tygodniach następujących po wybranym okresie treningowym. Użyj modelu regresji liniowej, gdzie zmienną niezależną będzie czas, a zmienną zależną:

### Dane treningowe i testowe

In [ ]:
def get_train_test(column, country, start, end):
    return part6.loc[(part6['country_name'] == country) & (part6['date'].between('2020-06-29', '2021-07-04')), ['date', column]],  part6.loc[(part6['country_name'] == country) & (part6['date'].between(start, end)), ['date', column]]

### Model regresji liniowej

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import datetime as dt


def linear_regression(train, test, ylabel, column):    
    reg = LinearRegression(n_jobs=-1)
    reg.fit(pd.DataFrame(train['date'].map(dt.datetime.toordinal)), train[column])
    
    y_pred = reg.predict(pd.DataFrame(test['date'].map(dt.datetime.toordinal)))
    
    print(f'Mean squared error: {mean_squared_error(test[column], y_pred)}')
    print(f'Coefficients: {reg.coef_}')
    print(f'Intercept: {reg.intercept_}')
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=test['date'], y=y_pred.squeeze(), label='Predicted', color='red')
    sns.scatterplot(x=test['date'], y=test[column].squeeze(), label='Actual', s=7)

    plt.xlabel('date')
    plt.ylabel(ylabel)
    plt.title(f'Model regresji liniowej dla {ylabel.lower()}')
    plt.legend()

### 3.1. liczba nowych zachorowań, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_confirmed', 'United States of America', '2021-12-28', '2022-07-04')
linear_regression(X_train, X_test, 'Liczba nowych zachorowań', 'daily_confirmed')

### 3.2. liczba nowych śmierci, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_deceased', 'United States of America', '2021-12-28', '2022-07-04')
linear_regression(X_train, X_test, 'Liczba nowych śmierci', 'daily_deceased')

### 3.3. liczba nowych szczepień, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_persons_vaccinated', 'United States of America', '2021-12-28', '2022-07-04')
linear_regression(X_train, X_test, 'Liczba nowych szczepień', 'daily_persons_vaccinated')

### 3.4 liczba nowych zachorowań mężczyzn, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_confirmed_male', 'United States of America', '2021-12-28', '2022-07-04')
linear_regression(X_train, X_test, 'Liczba nowych zachorowań meżczyzn', 'daily_confirmed_male')

### 3.4 liczba nowych śmierci mężczyzn, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_deceased_male', 'United States of America', '2021-12-28', '2022-07-04')
linear_regression(X_train, X_test, 'Liczba nowych śmierci mężczyzn', 'daily_deceased_male')

## <center>Część 4</center>
### Użyj model regresji wielomianowej dla wszystkich 5 przypadków rozważanych w części 3 poziomu 3. Oceń, który model regresji (liniowy czy wielomianowy) okazał się "lepszy" dla każdego przypadku. W tym celu możesz użyć dowolnej miary dopasowania modelu. Uzasadnij swoje zdanie.

In [ ]:
def polymonial_regression(train, test, ylabel, column):
    reg = np.poly1d(np.polyfit(train['date'].map(dt.datetime.toordinal), train[column], 3))
    
    y_pred = reg(test['date'].map(dt.datetime.toordinal))
    
    print(f'Mean squared error: {mean_squared_error(test[column], y_pred)}')
    print(f'Coefficients: {reg.c}')
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=test['date'], y=y_pred.squeeze(), label='Predicted', color='red')
    sns.scatterplot(x=test['date'], y=test[column].squeeze(), label='Actual', s=7)

    plt.xlabel('date')
    plt.ylabel(ylabel)
    plt.title(f'Model regresji liniowej dla {ylabel.lower()}')
    plt.legend()

### 4.1. liczba nowych zachorowań, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_confirmed', 'United States of America', '2021-12-28', '2022-07-04')
polymonial_regression(X_train, X_test, 'Liczba nowych zachorowań', 'daily_confirmed')

### 4.2. liczba nowych śmierci, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_deceased', 'United States of America', '2021-12-28', '2022-07-04')
polymonial_regression(X_train, X_test, 'Liczba nowych śmierci', 'daily_deceased')

### 4.3. liczba nowych szczepień, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_persons_vaccinated', 'United States of America', '2021-12-28', '2022-07-04')
polymonial_regression(X_train, X_test, 'Liczba nowych szczepień', 'daily_persons_vaccinated')

### 4.4 liczba nowych zachorowań mężczyzn, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_confirmed_male', 'United States of America', '2021-12-28', '2022-07-04')
polymonial_regression(X_train, X_test, 'Liczba nowych zachorowań meżczyzn', 'daily_confirmed_male')

### 4.4 liczba śmierci zachorowań mężczyzn, dla USA

In [ ]:
X_train, X_test = get_train_test('daily_deceased_male', 'United States of America', '2021-12-28', '2022-07-04')
polymonial_regression(X_train, X_test, 'Liczba nowych śmierci mężczyzn', 'daily_deceased_male')